In [ ]:
### Import
import os
import sys
import numpy as np
import logging
from dotenv import load_dotenv
from typing import cast

# Configuration logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Load environment variables from .env file
load_dotenv()

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_mistralai import MistralAIEmbeddings, ChatMistralAI
from langchain_community.vectorstores import FAISS
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

sys.path.insert(0, '/Users/xaviercoulon/Documents/OC/OC_P7_POC_RAG')
from openagenda_fetch import fetch_all_events, BASE_URL
from utils import event_to_langchain_document, invoke_rag_with_retry, invoke_with_intent_routing
from query_classifier import classify_query_intent, INTENT_RAG, INTENT_CHAT

# Configuration API
MISTRAL_API_KEY = cast(str, os.getenv("MISTRAL_API_KEY"))
if not MISTRAL_API_KEY:
	raise ValueError("MISTRAL_API_KEY environment variable not set")

print("✅ Tous les imports chargés avec succès")

✅ Tous les imports chargés avec succès


In [2]:
# Fetch all events et créer les LangChain Documents optimisés
events = fetch_all_events(BASE_URL)
print(f"Total events fetched: {len(events)}")

# Convertir les events en LangChain Documents optimisés
documents = [event_to_langchain_document(event) for event in events]
print(f"Total documents created: {len(documents)}")
print(f"Sample metadata keys: {list(documents[0].metadata.keys())}")


Total events fetched: 699
Total documents created: 699
Sample metadata keys: ['uid', 'slug', 'canonicalurl', 'location_city', 'location_department', 'location_region', 'location_address', 'location_postalcode', 'firstdate_begin', 'firstdate_end', 'lastdate_begin', 'lastdate_end', 'originagenda_title', 'originagenda_uid', 'age_min', 'age_max']


In [3]:
# Chunking, embedding et FAISS avec LangChain - Optimal setup

splitter = RecursiveCharacterTextSplitter(
	chunk_size=500,
	chunk_overlap=50,
	separators=["\n\n", "\n", ".", " ", ""],
)

# Split les documents
split_documents = splitter.split_documents(documents)
print(f"Total chunks created: {len(split_documents)}")

# Initialiser les embeddings Mistral
embeddings = MistralAIEmbeddings(
	model="mistral-embed",
	api_key=MISTRAL_API_KEY
)

# 3. Vector store FAISS (cosine auto-géré)
vector_store = FAISS.from_documents(
    documents=split_documents,
    embedding=embeddings,
    distance_strategy="COSINE",  # rend ton flow propre et explicite
)

print("✓ FAISS index created")
print("✓ Distance: cosine similarity (0-1)")
print(f"✓ Total vectors: {vector_store.index.ntotal}")

Total chunks created: 1629


/Users/xaviercoulon/Documents/OC/OC_P7_POC_RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO: HTTP Request: HEAD https://huggingface.co/mistralai/Mixtral-8x7B-v0.1/resolve/main/tokenizer.json "HTTP/1.1 307 Temporary Redirect"
INFO: HTTP Request: HEAD https://huggingface.co/mistralai/Mixtral-8x7B-v0.1/resolve/main/tokenizer.json "HTTP/1.1 307 Temporary Redirect"
INFO: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/mistralai/Mixtral-8x7B-v0.1/fc7ac94680e38d7348cfa806e51218e6273104b0/tokenizer.json "HTTP/1.1 200 OK"
INFO: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/mistralai/Mixtral-8x7B-v0.1/fc7ac94680e38d7348cfa806e51218e6273104b0/tokenizer.json "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 429 Too Many Re

✓ FAISS index created
✓ Distance: cosine similarity (0-1)
✓ Total vectors: 1629


In [4]:
# Recherche sémantique avec LangChain - Cosine similarity optimisée
query = "Quels événements sur la cuisine à Bayonne ?"

# Recherche de similarité avec cosine (normalisée)
results = vector_store.similarity_search_with_score(query, k=5)

print(f"\nTop 5 results for query: '{query}'")

for i, (doc, score) in enumerate(results):
    similarity_percent = (
        1 - score
    ) * 100  # score = distance → on convertit pour lisibilité

    print(f"--- Result {i+1} (Cosine Similarity: {similarity_percent:.1f}%) ---")
    print(f"Title: {doc.metadata.get('slug')}")
    print(f"City: {doc.metadata.get('location_city')}")
    print(f"Department: {doc.metadata.get('location_department')}")
    print(f"Date: {doc.metadata.get('firstdate_begin')}")
    print(f"Content: {doc.page_content[:250]}...\n")

INFO: HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO: HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO: HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 429 Too Many Requests"
INFO: HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"



Top 5 results for query: 'Quels événements sur la cuisine à Bayonne ?'
--- Result 1 (Cosine Similarity: 55.3%) ---
Title: le-secteur-de-lhotellerie-restauration-vous-interesse-participez-a-latelier-de-detection-de-potentiels-9363036
City: Bayonne
Department: Pyrénées-Atlantiques
Date: 2025-03-13 14:15:00+00:00
Content: . A l'issue de cette détection de potentiel vous aurez l’opportunité de rencontrer les acteurs du secteur lors du job dating Hôtellerie Café Restaurant le même jour à la CCI de Bayonne – Pays Basque Rendez-vous à la CCI Bayonne Pays Basque le 13/03 à...

--- Result 2 (Cosine Similarity: 55.3%) ---
Title: le-secteur-de-lhotellerie-restauration-vous-interesse-participez-a-latelier-de-detection-de-potentiels-6108864
City: Bayonne
Department: Pyrénées-Atlantiques
Date: 2025-03-13 14:15:00+00:00
Content: . A l'issue de cette détection de potentiel vous aurez l’opportunité de rencontrer les acteurs du secteur lors du job dating Hôtellerie Café Restaurant le même jour à la CCI

In [5]:
# Setup LLM Mistral et RAG chain
llm = ChatMistralAI(
    api_key=MISTRAL_API_KEY, model_name="mistral-small-latest", temperature=0.3
)

# Définir le prompt - AMÉLIORÉ pour mieux gérer les cas limites
template = """Tu es un assistant expert en événements français.

IMPORTANT:
- Utilise TOUT le contexte fourni pour répondre
- Si tu trouves des événements partiellement pertinents, mentionne-les quand même
- Sois exhaustif dans tes réponses
- Propose des événements similaires si l'exact n'existe pas

Contexte fourni (événements):
{context}

Question de l'utilisateur: {input}

Réponds en détail sur la base du contexte. Si vraiment aucun événement ne correspond, dis "Je n'ai pas trouvé d'événement exactement correspondant, mais voici ce qui pourrait vous intéresser..." et suggère les plus proches."""

prompt = ChatPromptTemplate.from_template(template)

# Créer la chaîne de combinaison des documents
stuff_chain = create_stuff_documents_chain(llm, prompt)

# Créer la chaîne RAG complète - AMÉLIORÉ : augmenter k à 5 ou 6
rag_chain = create_retrieval_chain(
    retriever=vector_store.as_retriever(search_kwargs={"k": 6}),  # ← Augmenté de 3 à 6
    combine_docs_chain=stuff_chain
)

print("✓ RAG chain créée et prête")
print("  - Retriever: k=6 chunks (augmenté pour plus de contexte)")
print("  - Prompt: Assouplissé pour accepter événements proches")

✓ RAG chain créée et prête
  - Retriever: k=6 chunks (augmenté pour plus de contexte)
  - Prompt: Assouplissé pour accepter événements proches


In [6]:
# Test du routing intelligent (RAG vs CHAT)
print("🎯 Test Routing Intelligent\n")

test_queries = [
    ("Bonjour, comment allez-vous ?", "CHAT"),
    ("Quels événements sur la cuisine à Bayonne ?", "RAG"),
    ("Merci beaucoup !", "CHAT"),
    ("Y a-t-il des concerts en novembre ?", "RAG"),
]

for query, expected_intent in test_queries:
    print(f"❓ Question: {query}")
    print(f"   Intent attendu: {expected_intent}")
    
    detected_intent = classify_query_intent(query, llm)
    print(f"   Intent détecté: {detected_intent}")
    print(f"   Status: {'✅' if detected_intent == expected_intent else '⚠️'}\n")

INFO: Classification de la requête: 'Bonjour, comment allez-vous ?...'


🎯 Test Routing Intelligent

❓ Question: Bonjour, comment allez-vous ?
   Intent attendu: CHAT


INFO: HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: Intention détectée: CHAT
INFO: Classification de la requête: 'Quels événements sur la cuisine à Bayonne ?...'
INFO: Intention détectée: CHAT
INFO: Classification de la requête: 'Quels événements sur la cuisine à Bayonne ?...'


   Intent détecté: CHAT
   Status: ✅

❓ Question: Quels événements sur la cuisine à Bayonne ?
   Intent attendu: RAG


INFO: HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: Intention détectée: RAG
INFO: Classification de la requête: 'Merci beaucoup !...'
INFO: Intention détectée: RAG
INFO: Classification de la requête: 'Merci beaucoup !...'


   Intent détecté: RAG
   Status: ✅

❓ Question: Merci beaucoup !
   Intent attendu: CHAT


INFO: HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: Intention détectée: CHAT
INFO: Classification de la requête: 'Y a-t-il des concerts en novembre ?...'
INFO: Intention détectée: CHAT
INFO: Classification de la requête: 'Y a-t-il des concerts en novembre ?...'


   Intent détecté: CHAT
   Status: ✅

❓ Question: Y a-t-il des concerts en novembre ?
   Intent attendu: RAG


INFO: HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: Intention détectée: RAG
INFO: Intention détectée: RAG


   Intent détecté: RAG
   Status: ✅



In [7]:
# Test du chatbot RAG avec gestion des erreurs 429
print("🤖 Chatbot RAG - Test avec gestion rate limit\n")

queries = [
    "Quels événements sur la cuisine à Bayonne ?",
    "Trouve-moi des événements agricoles",
    "Y a-t-il des concerts en novembre ?"
]

for query in queries:
	print(f"❓ Question: {query}")
	result = invoke_rag_with_retry(rag_chain, query, max_retries=3, initial_delay=2)
	print(f"✓ Réponse:\n{result['answer']}\n")
	print("-" * 80 + "\n")

🤖 Chatbot RAG - Test avec gestion rate limit

❓ Question: Quels événements sur la cuisine à Bayonne ?
🔄 Tentative 1/3...


INFO: HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


✓ Réponse:
D'après le contexte fourni, voici les événements liés à la cuisine et à la gastronomie à Bayonne :

1. **Foire au Jambon de Bayonne**
   - **Description** : Événement gourmand mettant en avant le "Jambon de Bayonne", un produit emblématique du territoire. Les restaurateurs sont encouragés à le valoriser, et la foire célèbre le savoir-faire artisanal et l'identité culinaire locale.
   - **Thème** : Gastronomie, produits locaux, patrimoine culinaire.

2. **Job Dating Hôtellerie Café Restaurant**
   - **Description** : Organisé à la CCI de Bayonne le **13/03** (15h15-16h30 pour les exercices, puis job dating à partir de 16h30). Bien que centré sur l'emploi, cet événement rassemble des acteurs du secteur de la restauration, ce qui peut être pertinent pour découvrir des opportunités ou des tendances culinaires locales.

3. **Visite guidée de Bayonne (thème "chocolat")**
   - **Description** : Les élèves de Bac Pro Métiers de l'Accueil et du Tourisme proposent une visite incluant 

INFO: HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


✓ Réponse:
Voici les événements agricoles pertinents basés sur le contexte fourni :

### **1. Animations agricoles et concours**
- **Événement** : Animations pour petits et grands, concours de fourrage et de fromage, exposition sur les sonnailles des troupeaux, bal ossalois.
- **Lieu** : Non précisé (mais probablement dans le Béarn ou les Pyrénées).
- **Catégories** : Élevage, agroalimentaire, traditions rurales.
- **Détails** : Idéal pour découvrir les métiers de l'élevage et les produits locaux.

### **2. Information collective sur les métiers agricoles**
- **Événement** : "Les métiers de l'élevage, de l'agroalimentaire, l'aménagement paysager, etc."
- **Date/Heure** : Jeudi 11 septembre à 10h.
- **Lieu** : ERIP du Béarn, site de Nay.
- **Catégories** : Élevage, agroalimentaire, installation agricole, aménagement paysager.
- **Détails** : Gratuit, ouvert à tous. Contact : 09 70 72 01 63 / eripbearnpau@mljpau.

### **3. Découverte des métiers agricoles (viticulture, agroalimentaire, é

INFO: HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"


✓ Réponse:
D'après le contexte fourni, voici les événements en novembre qui pourraient vous intéresser :

1. **Concert du Chœur Arraya** (sans date précise, mais dans le cadre des visites libres de 10h à 17h) :
   - Ce concert clôture une journée et se déroule dans un lieu historique (chapelle de l'ancien couvent des Franciscains).
   - Il met en avant de jeunes artistes avec un programme mêlant classique et découvertes musicales.
   - La participation est libre, ce qui en fait un événement accessible.

2. **Série de 6 concerts de l’Académie Ravel** (dans le cadre des Journées Européennes du Patrimoine, mais sans précision sur le mois) :
   - Ces concerts mettent en avant des jeunes talents et se déroulent dans des lieux patrimoniaux du Pays basque.
   - Certains pourraient avoir lieu en novembre, notamment des formats courts ou impromptus (gratuits ou à tarif modéré).

3. **Concert poétique avec instruments rares** (sans date précise, mais en partenariat avec l’Agora - Billère) :
   -

In [8]:
# Diagnostic: Voir ce que le retriever trouve pour "concerts"
print("🔍 Diagnostic Retriever - Cherchant 'concerts en janvier'\n")

diagnostic_query = "Y a-t-il eu des concerts en janvier ?"
retrieved_docs = vector_store.similarity_search_with_score(diagnostic_query, k=6)

print(f"Documents retrouvés par le retriever (k=6):\n")
for i, (doc, score) in enumerate(retrieved_docs):
    similarity = (1 - score) * 100
    print(f"--- Document {i+1} (Similarité: {similarity:.1f}%) ---")
    print(f"Titre: {doc.metadata.get('slug')}")
    print(f"Ville: {doc.metadata.get('location_city')}")
    print(f"Contenu: {doc.page_content[:300]}...\n")

print("-" * 80)
print("💡 Si les documents ci-dessus contiennent des concerts,")
print("   c'est un problème de prompt (trop restrictif).")
print("   Si pas de concerts du tout, c'est un problème de recherche.")
print("-" * 80 + "\n")

🔍 Diagnostic Retriever - Cherchant 'concerts en janvier'



INFO: HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"


Documents retrouvés par le retriever (k=6):

--- Document 1 (Similarité: 48.8%) ---
Titre: journees-europeennes-du-patrimoine-les-jardins-du-chateau-bijou
Ville: Labastide-Villefranche
Contenu: . Visites libres de 10h à 17h La journée se clôturera par un concert du Chœur Arraya. Catégories:...

--- Document 2 (Similarité: 48.0%) ---
Titre: concert-festival-off-ravel
Ville: Saint-Palais
Contenu: . La soirée aura lieu au cœur de la chapelle de l'ancien couvent des Franciscains, un lieu chargé d’histoire et propice à l’émotion musicale. Ce concert mettra en lumière de jeunes artistes en devenir, tous passionnés et portés par l’excellence musicale. Ils interpréteront un programme riche et sens...

--- Document 3 (Similarité: 47.8%) ---
Titre: concert-des-jeunes-solistes-de-lacademie-ravel
Ville: Saint-Pée-sur-Nivelle
Contenu: . Ce temps fort mettra à l’honneur une sélection de jeunes talents issus des dernières éditions de l’Académie Ravel avec une série de 6 concerts programmés dans des l